# **[PP-YOLOV2保姆级教程]使用自定义数据集实现吸烟识别预测**

## **PP-YOLOV2介绍**

通过结合多项有效的改进，在COCO2017上将PP-YOLO的性能从45.9％mAP提升到49.5％mAP。称之为PP-YOLOv2。

在速度方面，PP-YOLOv2在640x640输入尺寸下以68.9FPS的速度运行。具有TensorRT，FP16精度和Batch=1的Paddle推理引擎进一步提高了PP-YOLOv2的推理速度，达到了106.5 FPS。这样的性能超过了具有大致相同数量的参数（即YOLOv4-CSP，YOLOv5l）的现有目标检测器。此外，带有ResNet101的PP-YOLOv2在COCO2017测试开发上达到了50.3％的mAP。

![](https://ai-studio-static-online.cdn.bcebos.com/9b582f8b16254aa285e03800894fa7f8142620dc61e241838a2ddae7d08ad58c)


## **手把手教你在AIStudio平台上使用PaddleDetection API训练自己的数据集**
**教程目的：** 通过PaddleDetection API熟悉经典的目标检测框架，为下一步自己手写实现目标检测网络打基础

**教程内容：** 以PaddleDetection API中的特色模型为例，介绍网络结构，以及如何使用此框架训练自己的数据集

**数据准备：** 本教程程基于pp_fall数据集，为方便读者体验，已经上传至 data/data94796/pp_smoke.zip。

**PaddelDetection：** 为方便读者体验，存放在PaddleDetection，使用PP-YOLOV2。

**其他说明：** 本教程所有命令均在Notebook中执行。


## **检测效果**

![](https://ai-studio-static-online.cdn.bcebos.com/802e7adbff0c4ddb885711c42ef2cf9fb61d53ed247b423891e7bd4cfb753f55)


## **做这个项目的缘由：**
看了许多PP-YOLOV2相关的项目，发现许多用来制作数据集都是COCO格式，这对于我这种习惯用VOC格式的人来说，就相对有点别扭，因此想自己弄一个VOC数据集格式的PP-YOLOV2，顺便熟悉一下PP-YOLOV2。


# **实验步骤：**

## **一、解压数据集**

In [ ]:
!unzip data/data94796/pp_smoke.zip

## **二、安装PaddleDetection包**

最新的PaddleDetection包我已经放在了/home/aistudio/data/data98540/PaddleDetection-release-2.1.zip，解压后，安装相关的模块即可

In [ ]:
!unzip -oq /home/aistudio/data/data98540/PaddleDetection-release-2.1.zip

In [1]:
!pip install -r /home/aistudio/PaddleDetection-release-2.1/requirements.txt

## **三、按比例划分数据集**
**ratio比例系数**

In [ ]:
import random
import os
#生成train.txt和val.txt
random.seed(2020)
xml_dir  = '/home/aistudio/Annotations'#标签文件地址
img_dir = '/home/aistudio/images'#图像文件地址
path_list = list()
for img in os.listdir(img_dir):
    img_path = os.path.join(img_dir,img)
    xml_path = os.path.join(xml_dir,img.replace('jpg', 'xml'))
    path_list.append((img_path, xml_path))
random.shuffle(path_list)
ratio = 0.9
train_f = open('/home/aistudio/work/train.txt','w') #生成训练文件
val_f = open('/home/aistudio/work/val.txt' ,'w')#生成验证文件

for i ,content in enumerate(path_list):
    img, xml = content
    text = img + ' ' + xml + '\n'
    if i < len(path_list) * ratio:
        train_f.write(text)
    else:
        val_f.write(text)
train_f.close()
val_f.close()

#生成标签文档
label = ['smoke']#设置你想检测的类别
with open('/home/aistudio/work/label_list.txt', 'w') as f:
    for text in label:
        f.write(text+'\n')

## **注意：**
后面跑训练的时候，提示了这个错误，查明原因发现是有一张照片有问题，于是就删除掉，后面跑就成功了。要删除smoke_a187.jpg照片以及其对应的标注文件

In [ ]:
%cd ~
import os
file_name = "Annotations/smoke_a187.xml"#/images//smoke_a187.jpg  /Annotations//smoke_a187.xml
if os.path.exists(file_name):
    os.remove(file_name)
    print('成功删除文件:', file_name)
else:
    print('未找到此文件:', file_name)

/home/aistudio
成功删除文件: Annotations/smoke_a187.xml


## **四、执行训练**

此次使用的是 PaddleDetection-release-2.1/configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml 文件，并开启vdl

## **修改部分：**

只需要修改对应的红色部分

PaddleDetection-release-2.1/configs/datasets/voc.yml


![](https://ai-studio-static-online.cdn.bcebos.com/e5fa5314d7d64eb49b3c0c5979c07cfb808bd150c3a14c7c9e8f6d39fb193d21)

如果爆内存就修改这两个文件的batch_size：

PaddleDetection-release-2.1/configs/ppyolo/_base_/ppyolov2_reader.yml

PaddleDetection-release-2.1/configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml


In [6]:
%cd ~/PaddleDetection-release-2.1/
!python tools/train.py -c configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml --eval --use_vdl=True --vdl_log_dir="./output"

![](https://ai-studio-static-online.cdn.bcebos.com/58996194619f478c80f458c10acb9e8c83f8e2d0013f450c9f1ff97f3d4058d5)


## **五、模型评估**

In [7]:
!python -u tools/eval.py -c configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml  -o weights=output/ppyolov2_r50vd_dcn_voc/best_model.pdparams

![](https://ai-studio-static-online.cdn.bcebos.com/02de61111cf24ccaac768ff72403a4136b53bed98ed94f6496ebf67b03047f22)


## **打开可视化**

步骤：

打开左侧可视化

![](https://ai-studio-static-online.cdn.bcebos.com/cbf8b5d4fb464ae1a2d773f5e36b00dcb134556b27b24a5ba5fe962331cb36d8)

设置logdir ： 路径选择PaddleDetection-release-2.1/output

选择模型文件：路径选择PaddleDetection-release-2.1/output/ppyolov2_r50vd_dcn_voc/best_model.pdparams

然后按下面的打开VisualDL

即可看到如图

![](https://ai-studio-static-online.cdn.bcebos.com/40d84bb78ff64f968f0f63e6392050c68b5a68421ebf4275a7292a78a9434aa2)

这是我训练了100epochs的结果，如果接下去训练mAP应该会更高。

还有下图的参数记录

![](https://ai-studio-static-online.cdn.bcebos.com/64beb062f138470a890d3de1774113aaac1b216184354cd889e079f7699e12c8)




## **六、模型预测**
在执行tools/infer.py后，在output文件夹下会生成对应的预测结果

In [7]:
%cd ~/PaddleDetection-release-2.1/
!python tools/infer.py -c configs/ppyolo/ppyolov2_r50vd_dcn_voc.yml -o weights=output/ppyolov2_r50vd_dcn_voc/best_model.pdparams --infer_img=/home/aistudio/smoke1.png

/home/aistudio/PaddleDetection-release-2.1
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/__init__.py:107: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import MutableMapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/rcsetup.py:20: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Iterable, Mapping
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/matplotlib/colors.py:53: DeprecationWarning: Using or importing the ABCs from 'collections' instead of from 'collections.abc' is deprecated, and in 3.8 it will stop working
  from collections import Sized
/opt/conda/envs/python35-paddle120-env/lib/python3.7/site-packages/paddle/tensor/creation.py:125: DeprecationWarning

## **结果展示**


**原图**

![](https://ai-studio-static-online.cdn.bcebos.com/41ff1e9c3bad4c7c840bd0c9c89898cf5db63094f5754a5281ca403e7652fa49)




**预测图**

![](https://ai-studio-static-online.cdn.bcebos.com/282ee4e0896c42d5a29b047a31327598d648c9076f7d42389772ed09e7e58bf7)




# **总结**

由上图我们可以看到，使用PP-YOLOV2完成了吸烟的目标识别检测，并且mAP已经达到了86.74%。

优化方案

可以通过增加数据集、选择更优化模型，增加训练的次数。

后期应用

后期可以部署到商场、医院、疗养院等公共场合，进行禁烟管控。

![](https://ai-studio-static-online.cdn.bcebos.com/e8836f09b4654eaba7ef4e941cb8573c9b64d3b4d20c405680273a1f3fad9ac7)


# **关于更多关于PaddleDetection的信息请参考下面的链接地址。**

PaddleDetection教程文档地址：https://github.com/PaddlePaddle/PaddleDetection

PaddleDetection Github地址：https://github.com/PaddlePaddle/PaddleDetection


# **关于作者**

感兴趣的方向为：目标检测，分类任务等

AIstudio主页： 来互关呀~ 我在AI Studio上获得黄金等级，点亮8个徽章，来互关呀~ https://aistudio.baidu.com/aistudio/personalcenter/thirdview/474269

Github主页： https://github.com/Niki173

欢迎大家有问题留言交流学习，共同进步成长。